In [98]:
#Update your token
STUDENT_TOKEN = 'GABRIEL DE OLAGUIBEL'

## ignore this code, just used for submission
import requests
import pprint
import json
import random
import math
from copy import copy, deepcopy

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)

In [99]:
def format_value(value):
  if value == '.':
    return '..'
  else:
    s = str(value)
    if len(s) == 1:
      s = ' ' + s
    return s

def format_h(const, max):
  s = ''
  for i in range(max - len(const)):
    s = s + '   '
  s = s + ' '.join(map(format_value, const))
  return s

def pretty_print(board, h_const, v_const):
  max_h = 1
  for c in h_const:
    max_h = max(max_h, len(c))
  max_v = 1
  for c in v_const:
    max_v = max(max_v, len(c))

  h_offset = ' '.join(map(lambda x: '  ', range(max_h)))
  for i in range(max_v):
    print('    ' + h_offset + ' '.join(map(lambda x: '  ' if i >= len(x) else format_value(x[i]), v_const)))
  for r in range(len(board)):
    row = board[r]
    print((h_offset if r ==0 else format_h(h_const[r-1], max_h)) + ' ' +  ' '.join(map(format_value, row)))

# Kakuro

In [100]:
# Board representation
class Kakuro:
    def __init__(self, puzzle):
        self.board = puzzle['board']
        self.h_constraints = puzzle['h_constraints']
        self.v_constraints = puzzle['v_constraints']
        self.id = puzzle.get('id', None)

    # Ill probably need to change this for the hard problems f

    def check_horizontal(self, row, col):
        target_sum = self.h_constraints[row-1][0]
        total, count = 0, 0
        for c in range(col, len(self.board[row])): 
            cell = self.board[row][c]
            if cell == 'X':  # means it the end of th sequence
                break
            if cell == '.':  # find empty cell
                continue
            total += int(cell) # suming the values
            count += 1
        if count == 0:  # cells filled in yet
            return True
        if total > target_sum:
            return False  # Sum exceeded so it is not valid
        if 'X' not in self.board[row][col:] and total != target_sum: # checking if it is not the target sum
            return False  
        return True

    # follows a similar approach to the horizontal check
    def check_vertical(self, row, col):
        target_sum = self.v_constraints[col-1][0] 
        total, count = 0, 0
        for r in range(row, len(self.board)):
            cell = self.board[r][col]
            if cell == 'X':  
                break
            if cell == '.':  
                continue
            total += int(cell)
            count += 1
        if count == 0:  
            return True
        if total > target_sum:
            return False  
        if 'X' not in [self.board[r][col] for r in range(row, len(self.board))] and total != target_sum:
            return False  
        return True

    def is_valid(self):
        # Check horizontal constraints
        for r, row in enumerate(self.board): 
            for c, value in enumerate(row):
                if value != 'X' and value != '.': # i'll Check only cells with values
                    # Check the sum of horizontal sequence using the function
                    if not self.check_horizontal(r, c): 
                        return False
                    # now checking vertical
                    if not self.check_vertical(r, c):
                        return False
        return True
    




### Constraint propagation

In [103]:
import itertools # using this library to generate all possible combinations of a list which is useful when applying constraints


# function to Determine the possible values for each empty cell in a sequence by considering all other cells.
def generate_possible_values(empty_indices, filled_cells, target_sum): 

    remaining_sum = target_sum - sum(filled_cells)
    
    # Exclud numbers already used in the filled cells when generating combinations
    available_numbers = [num for num in range(1, 10) if num not in filled_cells]
    
    # Generate all possible combinations of numbers that can fill the empty cells and match the remaining sum.
    valid_permutations = [seq for seq in itertools.permutations(available_numbers, len(empty_indices)) 
                          if sum(seq) == remaining_sum and all(num not in filled_cells for num in seq)] # checking if the numbers are not in the filled cells
    
    # For each cell position, determine the possible values it can take
    possible_values_per_cell = [set() for _ in empty_indices] 
    for perm in valid_permutations:
        for i, val in enumerate(perm): # i is index and val is obviously the value btw
            possible_values_per_cell[i].add(val)
    
    return possible_values_per_cell 

# function provides a sequence (horizontal or vertical) that the cell belongs to and its sum constraint
# We will redefine the find_sequence_and_sum function with additional checks and debug output
def find_sequence_and_sum(board, row, col, constraints, is_horizontal):
    if is_horizontal:
        start, end = col, col 
        while start > 0 and board[row][start - 1] != 'X': # checking if the cell were looking at is not the first one in the sequence
            start -= 1 # move left to find it
        while end < len(board[row]) and board[row][end] != 'X': # checking if the cell were looking at is not the last one in the sequence
            end += 1 # move right to find it
        sequence = board[row][start:end] 
        target_sum = constraints[row - 1][0] if row > 0 else None # if the row is 0 then there is no constraint
    else: # same thing but for vertical:
        start, end = row, row
        while start > 0 and board[start - 1][col] != 'X':
            start -= 1
        while end < len(board) and board[end][col] != 'X':
            end += 1
        sequence = [board[i][col] for i in range(start, end)]
        target_sum = constraints[col - 1][0] if col > 0 else None # if the col is 0 then there is no constraint

    return sequence, target_sum

# function updates the board with the possible values for each sequence given the filled cells and the sum constraint
def propagate_constraints(board, row, col, h_constraints, v_constraints): 
    # Find the sequemces affected by the placement of a number 
    h_word, h_target_sum = find_sequence_and_sum(board, row, col, h_constraints, True)
    v_word, v_target_sum = find_sequence_and_sum(board, row, col, v_constraints, False)

    # Initialize possibilities for the horizontal and vertical sequences
    # if the cell is empty then it can be any number from 1 to 9 initially
    h_possibilities = [set(range(1, 10)) if x == '.' else {int(x)} for x in h_word]
    v_possibilities = [set(range(1, 10)) if x == '.' else {int(x)} for x in v_word]

    # use generate_possible_values for the empty cells in the horizontal and vertical sequences
    if h_target_sum: # if there is a constraint
        h_empty_indices = [i for i, x in enumerate(h_word) if x == '.'] # geting the index of the empty cells
        h_filled_cells = [int(x) for x in h_word if x != '.'] # getting the filled cells
        h_possibilities_for_empty = generate_possible_values(h_empty_indices, h_filled_cells, h_target_sum) # calling the function
        for i, index in enumerate(h_empty_indices):
            h_possibilities[index] = h_possibilities_for_empty[i]

    if v_target_sum:
        v_empty_indices = [i for i, x in enumerate(v_word) if x == '.']
        v_filled_cells = [int(x) for x in v_word if x != '.']
        v_possibilities_for_empty = generate_possible_values(v_empty_indices, v_filled_cells, v_target_sum)
        for i, index in enumerate(v_empty_indices):
            v_possibilities[index] = v_possibilities_for_empty[i]

    # Get the index of the current cell in the horizontal and vertical sequences
    h_index = h_word.index('.') if '.' in h_word else None
    v_index = v_word.index('.') if '.' in v_word else None

    # Combine the possibilities for the current cell from the horizontal and vertical sequences
    combined_possibilities = set(range(1, 10)) # intital possibilities
    if h_index is not None and h_index < len(h_possibilities): 
        combined_possibilities.intersection_update(h_possibilities[h_index]) # using intersection_update which updates the set with the intersection of itself and another 
    if v_index is not None and v_index < len(v_possibilities):
        combined_possibilities.intersection_update(v_possibilities[v_index]) 

    # Return the combined possibilities for the current sequence
    return combined_possibilities 

# this function returns the possible values for a cell by combining the constraints from the horizontal and vertical sequences
def get_possible_values(board, row, col, h_constraints, v_constraints):
    possible_values = set(range(1, 10)) # intital possibilities
    # Propagate constraints from the current board state
    combined_possibilities = propagate_constraints(board, row, col, h_constraints, v_constraints)
    # The combined possibilities are for the current cell only
    possible_values.intersection_update(combined_possibilities)
    return possible_values # this is different from the one in the previous function because this one is for a single


### Minimum remaining values implementation

In [104]:
# heuristic function to determine the cell with the minimum remaining values for efficientcy

def calculate_mrv(board, h_constraints, v_constraints):
    mrv_cells = [] 
    # find all empty cells
    for row in range(len(board)):
        for col in range(len(board[0])):
            if board[row][col] == '.':
                # Get the possible values for each empty cell
                possible_values = get_possible_values(board, row, col, h_constraints, v_constraints)
                mrv_cells.append(((row, col), len(possible_values))) # add it to the list with the length of the possible values
    # Sort by the number of possible values, which is the second item in the tuples
    mrv_cells.sort(key=lambda item: item[1])
    '''
    # used this to show the sorted MRV cells for each step to make sure its working
    print(f"MRV cells sorted by possible values: {mrv_cells}")
    '''
    return mrv_cells

### Solver

In [105]:
def solve_kakuro(board, h_constraints, v_constraints):

    backtracks = 0  

    # Initialize MRV with the first call to calculate_mrv.
    mrv_cells = calculate_mrv(board, h_constraints, v_constraints)
    if not mrv_cells: # edge case
        print("No empty cells found, puzzle is already solved or invalid.")
        return board, backtracks

    # Start with the cell with the minimum remaining values (first in the list)
    (row, col), _ = mrv_cells[0] 
    #print(f"Starting with cell ({row}, {col}) with minimum remaining values.") # used this to check if it was working

    # Initialize the stack with the first cell
    stack = [(board, row, col)] 

    while stack:
        board, row, col = stack.pop()  # Get the current state and cell to fill
        '''
        print(f"\nCurrent state: {board}")
        print(f"Current position: ({row}, {col})")
        print(f"Stack size: {len(stack)}, Backtracks: {backtracks}") 
        '''
        # Get possible values for the current cell
        possible_values = get_possible_values(board, row, col, h_constraints, v_constraints)

        # If there are no possible values, we backtrack
        if not possible_values:
            backtracks += 1
            # print("No possible values, backtracking...")
            continue  # Backtrack to the previous state. I only need continue here because the stack already has the previous state 

        # Try each possible value and push the new state onto the stack
        for value in possible_values:
            new_board = [list(r) for r in board]  # Create a deep copy of the board
            new_board[row][col] = str(value)  # Assign the value as a string
            #print(f"Trying value {value} at cell ({row}, {col})")

            # Update MRV after the assignment
            next_mrv_cells = calculate_mrv(new_board, h_constraints, v_constraints)
            if next_mrv_cells: 
                next_row, next_col = next_mrv_cells[0][0] # Get the next cell with minimum remaining values
                #print(f"Next cell to try is ({next_row}, {next_col}) with MRV.")
                stack.append((new_board, next_row, next_col))  # Push the new state with MRV cell
            else:
                # If MRV list is empty, it means all cells are filled and passed all constraints
                #print("Solution found")
                return new_board, backtracks

    # If the stack was empty, no solution was found
    print("No solution found.")
    return None, backtracks

### Test To Server

In [112]:
# Get the problem from the server
problem = get_problem('kakuro')

# print what it got from the server
pprint.pprint(problem)
print("\n")

# extract the board and constraints
board = problem['board']
h_const = problem['h_constraints']
v_const = problem['v_constraints']

# print the initial board nicely
print('Input puzzle')
pretty_print(board, h_const, v_const)
print("\n")

# Solve the puzzle 
solution, backtracks = solve_kakuro(board, h_const, v_const)
solution, backtracks

# print the solution nicely
print('\nSolution:')
pretty_print(solution, problem['h_constraints'], problem['v_constraints'])

answer = {
    'board': solution,
    'h_const': h_const,
    'v_const': v_const,
    'id': problem['id']
}

submit_answer('kakuro', answer)

# print the number of backtracks
print(f"Backtracks: {backtracks}")

{'board': [['X', 'X', 'X', 'X', 'X'],
           ['X', '.', '.', 'X', 'X'],
           ['X', '.', '.', '.', 'X'],
           ['X', 'X', '.', '.', '.'],
           ['X', 'X', 'X', '.', '.']],
 'h_constraints': [[11], [14], [20], [17]],
 'id': 3,
 'v_constraints': [[16], [7], [23], [16]]}


Input puzzle
      16  7 23 16
    X  X  X  X  X
11  X .. ..  X  X
14  X .. .. ..  X
20  X  X .. .. ..
17  X  X  X .. ..



Solution:
      16  7 23 16
    X  X  X  X  X
11  X  9  2  X  X
14  X  7  1  6  X
20  X  X  4  9  7
17  X  X  X  8  9
Ignore: not checking values yet
Backtracks: 0


Manually test hard

In [106]:
problem = {'board': [['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'],
           ['X', '.', '.', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'],
           ['X', '.', '.', '.', 'X', '.', '.', 'X', 'X', 'X', 'X', '.', '.'],
           ['X', 'X', 'X', '.', '.', '.', '.', 'X', 'X', '.', '.', '.', '.'],
           ['X', 'X', '.', '.', '.', '.', 'X', 'X', '.', '.', '.', 'X', 'X'],
           ['X', 'X', '.', '.', 'X', '.', '.', 'X', '.', '.', 'X', 'X', 'X'],
           ['X', 'X', 'X', '.', '.', 'X', '.', '.', 'X', '.', '.', 'X', 'X'],
           ['X', 'X', 'X', '.', '.', 'X', '.', '.', 'X', '.', '.', 'X', 'X'],
           ['X', 'X', 'X', 'X', '.', '.', 'X', '.', '.', 'X', '.', '.', 'X'],
           ['X', 'X', 'X', '.', '.', '.', 'X', 'X', '.', '.', '.', '.', 'X'],
           ['X', '.', '.', '.', '.', 'X', 'X', '.', '.', '.', '.', 'X', 'X'],
           ['X', '.', '.', 'X', 'X', 'X', 'X', '.', '.', 'X', '.', '.', '.'],
           ['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', '.', '.']],
 'h_constraints': [[11],
                   [12, 8, 12],
                   [22, 17],
                   [19, 18],
                   [17, 7, 16],
                   [12, 11, 9],
                   [15, 7, 11],
                   [13, 9, 5],
                   [15, 23],
                   [15, 14],
                   [3, 3, 14],
                   [9]],
 'id': 0,
 'v_constraints': [[3, 3],
                   [17, 16, 4],
                   [38, 3],
                   [3, 35],
                   [11, 16],
                   [16, 7],
                   [23, 3],
                   [16, 10],
                   [35, 17],
                   [4, 22],
                   [4, 4, 3],
                   [17, 17]]}

# print the initial board
pretty_print(problem['board'], problem['h_constraints'], problem['v_constraints'])

# Now, let's solve the puzzle again with the refined 'get_possible_values' function
solution, backtracks = solve_kakuro(problem['board'], problem['h_constraints'], problem['v_constraints'])
solution, backtracks

# print the solution
pretty_print(solution, problem['h_constraints'], problem['v_constraints'])

             3 17 38  3 11 16 23 16 35  4  4 17
             3 16  3 35 16  7  3 10 17 22  4 17
                4                          3   
          X  X  X  X  X  X  X  X  X  X  X  X  X
      11  X .. ..  X  X  X  X  X  X  X  X  X  X
12  8 12  X .. .. ..  X .. ..  X  X  X  X .. ..
   22 17  X  X  X .. .. .. ..  X  X .. .. .. ..
   19 18  X  X .. .. .. ..  X  X .. .. ..  X  X
17  7 16  X  X .. ..  X .. ..  X .. ..  X  X  X
12 11  9  X  X  X .. ..  X .. ..  X .. ..  X  X
15  7 11  X  X  X .. ..  X .. ..  X .. ..  X  X
13  9  5  X  X  X  X .. ..  X .. ..  X .. ..  X
   15 23  X  X  X .. .. ..  X  X .. .. .. ..  X
   15 14  X .. .. .. ..  X  X .. .. .. ..  X  X
 3  3 14  X .. ..  X  X  X  X .. ..  X .. .. ..
       9  X  X  X  X  X  X  X  X  X  X  X .. ..
No solution found.
             3 17 38  3 11 16 23 16 35  4  4 17
             3 16  3 35 16  7  3 10 17 22  4 17
                4                          3   


TypeError: object of type 'NoneType' has no len()